# DEMO notebook for the eft model
This demo showcases the implementation of the eft model for the anisotropic galaxy power spectrum.
The model implemented is the one of [2004.10607](https://arxiv.org/pdf/2004.10607.pdf), and features:
* 1-loop galaxy bias, with locan and non local contributions. The bias expansion assumed is an Eulerian one, with $\{b_1,~b_2,~b_{\gamma_2},~b_{\Gamma_3}\}$;
* 1-loop corrections, computed in an efficient way with a custom implementation based on the FastPT package [1603.04826](https://arxiv.org/pdf/1603.04826.pdf). These include EFT couterterms for the three multipoles $\{c_0,~c_2,~c_4\}$;
* a customized IR-resummation routine to account for the non-linear evolution of the BAO peak;
* a stochastic contribution to the power spectrum that includes a constant deviation from Poisson shot noise as well as scale dependent terms $\{\alpha_P,~\epsilon_{0,k^2},~\epsilon_{2,k^2}\}$.

The code takes in the cosmology dictionary and returns a 2D interpolator for the anisotropic power spectrum $P(k,\mu)$.

In [ ]:
# Import relevant packages
import numpy as np
import matplotlib.pyplot as plt
import time 
import os, sys
import copy
from scipy.special import legendre
import scipy.integrate as integ

In [ ]:
# Set path
likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

In [ ]:
# Matplotlib params set-up
%matplotlib inline
plt.rc('text', usetex=True)
plt.rc('font', size=20)
plt.rc('xtick', direction='in', labelsize=16)
plt.rc('ytick', direction='in', labelsize=16)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
font = {'family' : 'serif'}
plt.rc('font', **font)  # pass in the font dict as kwargs

In [ ]:
from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
info = {
'params': {
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns':0.965,
        'w': -1, #Dark energy fluid model
        'wa': 0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega barion density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    'sampler': {'evaluate': None},  
    'output': 'chains/my_euclid_experiment',
    'debug': True,
    'timing': True,
    'force': True,
}

In [ ]:
info['likelihood'] = {'Euclid': 
                         {'external': EuclidLikelihood, # Likelihood Class to be read as external
                         'observables_selection': {
                             'WL': {'WL': False, 'GCphot': False, 'GCspectro': False},
                             'GCphot': {'GCphot': False, 'GCspectro': False},
                             'GCspectro': {'GCspectro': True},
                             'CG': {'CG': False},
                             'add_phot_RSD': False,
                             'matrix_transform_phot' : False # 'BNT' or 'BNT-test'
                             },
                         'plot_observables_selection': True,  
                         'NL_flag_phot_matter': 4,
                         'NL_flag_phot_baryon': 0,
                         'NL_flag_phot_bias': 0,
                         'NL_flag_spectro': 0,
                         'IA_flag': 0,
                         'k_max_extrap': 500.0,
                         'k_min_extrap': 1E-5,
                         'k_samp': 1000,
                         # z limit values and size z-array
                         'z_min': 0.0,
                         'z_max': 4.0,
                         'z_samp': 100,
                         # Use MG gamma
                         'use_gamma_MG': False,
                         # Use Weyl bypass
                         'use_Weyl': False,
                         # Use redshift-dependent purity for GCspectro or not
                         'f_out_z_dep': False,
                         # Add spectroscopic redshift errors
                         'GCsp_z_err': True,
                         # Print theory predictions
                         'print_theory': False,
                         'use_magnification_bias_spectro': 0,
                         'use_Weyl': False,
                         # Plot the selected observables matrix
                         'plot_observables_selection': True,
                         'IR_resum': 'DST',
                         'use_magnification_bias_spectro': 0,
                         'Baryon_redshift_model': False,
                         'data': { 
                            'sample': 'ExternalBenchmark',
                            'spectro': {
                                'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                                'redshifts': ["1.", "1.2", "1.4", "1.65"],
                                'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                                'root_mixing_matrix': 'mm_FS230degCircle_m3_nosm_obsz_z0.9-1.1.fits',
                                'Fourier': True,
                                'scale_cuts_fourier': 'GCspectro-FourierSpace.yaml'},
                            'photo': {
                                'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                                'ndens_GC': 'niTab-EP10-RB00.dat',
                                'ndens_WL': 'niTab-EP10-RB00.dat',
                                'root_GC': 'Cls_{:s}_PosPos.dat',
                                'root_WL': 'Cls_{:s}_ShearShear.dat',
                                'root_XC': 'Cls_{:s}_PosShear.dat',
                                'IA_model': 'zNLA',
                                'Fourier': True,
                                'luminosity_ratio': 'luminosity_ratio.dat',
                                'photo_data': 'standard',
                                'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                                'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                                'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                                'cov_3x2': 'CovMat-3x2pt-{:s}-20Bins.npz',
                                'cov_model': 'Gauss'}
                         }                 
                        }
                     }

In [ ]:
from cobaya.model import get_model

from cloe.auxiliary.yaml_handler import yaml_write
yaml_write('test',info, overwrite=True)
# The `get_model` function of Cobaya imported in the line above needs a yaml or dictionary as an argument
# 
# We measure the time to give us an estimation of how much time it takes to make the initialization of the
# likelihood
t1 = time.time()

# Second: create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)
# We need to sample the log posterior to create model.provider.params
logposterior = model.logposterior({})

In [ ]:
# Create an instance of the class EuclidLikelihood and initialize it (general case)
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)

# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)

## Non-linear power spectrum

There are now two options to access the non-linear power spectrum:
1. **through the cosmology dictionary**: if the non-linear flag is set to `NL_flag=5`, the `Pgg_spectro` entry of the cosmology dictionary contains the eft non-linear Pgg. This is computed for a set of bias parameters specified in the `info` dictionary, in the `['params']` entry. If they are not specified, the code defaults to the `params.yaml` file in the `configs` folder, i.e. to the matter power spectrum ($b_1=1$, all other parameters set to 0).

2. **through the eft module**: in this case both the redshift and the bias parameters can be specified directly (the latter using a dictionary).

**NOTE**: the `Pgg_spectro` entry of the cosmology dictionary has HARDCODED redshifts. This is done to avoid a 3D interpolation (over $k$,$\mu$,$z$), but still have the `spectro` function redshift dependent as the linear one. The hardcoded redshifts are the ones of the IST:Forecast paper. This means that requesting a power spectrum at $z=0.95$, will result in a $P(k,\mu)$ at $z=1$, the center of the first redshift bins.

### Example 1: access through the cosmology dictionary

In [ ]:
# Check the non linear flag
print("Non-linear flag:",like.cosmo.cosmo_dic['NL_flag_phot_matter'])

In [ ]:
# Set up k, mu vectors and redshift values
kvec = np.logspace(-3,1,1000)
muvec = np.linspace(-1,1,101)
z0 = 0.0
z1 = 1.0

In [ ]:
# Select z=1 and put P(k,mu) in an array
zz = z1
pkmu = np.array(like.cosmo.cosmo_dic['Pgg_spectro'](zz, kvec, muvec[0]))
print("k, mu:",pkmu.shape)

For illustrative purposes here we do the projection to multipoles manually. In CLOE this is done by IST:L, as well as the inclusion of the Alcock-Paczynski effect.

In [ ]:
# Project to multipoles
pk_multipoles = np.zeros((3,len(kvec)))
for i in range(3):
    integrand=np.array([pkmu[j]*legendre(2*i)(muvec) for j in range(len(kvec))])
    pk_multipoles[i] = integ.simps(integrand, x=muvec, axis=1) * (4.*i+1)/2

In [ ]:
# Plot multipoles -- remember these are matter with no counterterm and no shot noise
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)
ax.set_title(r'$z={0}$'.format(zz))
plt.plot(kvec, pk_multipoles[0], label=r'$P_0$')
plt.plot(kvec, pk_multipoles[1], label=r'$P_2$')
plt.plot(kvec, pk_multipoles[2], label=r'$P_4$')
ax.set_ylabel(r"$P_{mm,\ell}$")
ax.set_xlabel(r'$k$')
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend()
plt.show()

### Example 2: eft module
Here I access the `P_kmu_z` function of the `eft` module directly, but don't specify the bias parameters, i.e. it's still the matter power spectrum, now computed at redshift $z=0$. This requires importing the `EFTofLSS` module.

In [ ]:
from cloe.non_linear.eft import EFTofLSS
zz = z0 # select redshift zero

# Initialize eftobject and compute loop corrections
eftobj = EFTofLSS(like.cosmo.cosmo_dic)
eftobj._Pgg_kmu_terms(zz)

# Construct anisotropic power spectrum
Pkmu = eftobj.P_kmu_z(zz, f=like.cosmo.cosmo_dic['f_z'](zz), D=like.cosmo.cosmo_dic['D_z_k_func'](zz, 1e-5),
                      **like.cosmo.cosmo_dic) # this is the k,mu interpolator
pkmu_nlmod = Pkmu(kvec, muvec) # compute over the kvec, muvec grid

In [ ]:
# Project to multipoles
pk_nlmod = np.zeros((3,len(kvec)))
for i in range(3):
    integrand=np.array([pkmu_nlmod[j]*legendre(2*i)(muvec) for j in range(len(kvec))])
    pk_nlmod[i] = integ.simps(integrand, x=muvec, axis=1) * (4.*i+1)/2

In [ ]:
# Plot
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)
ax.set_title(r'$z={0}$'.format(zz))
plt.plot(kvec, pk_nlmod[0], label=r'$P_0$')
plt.plot(kvec, pk_nlmod[1], label=r'$P_2$')
plt.plot(kvec, pk_nlmod[2], label=r'$P_4$')
ax.set_ylabel(r"$P_{mm,\ell}$")
ax.set_xlabel(r'$k$')
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend()
plt.show()

### Example 3: eft module + bias parameters
This is the same as Example 2, i.e. the non-linear power spectrum is accessed directly through the `eft` module, but with a full set of bias parameters.

In [ ]:
# Bias parameters are specified in a dictionary
biasdict_z1 = {'b1': 1.369988, 'b2': -0.513959, 'bG2': -0.105710, 'bG3': -0.34522775,
               'c0': 9.542350, 'c2': 11.390455, 'c4': 2.469497, 'cnlo': 12.972343,
               'aP': 0.315394, 'Psn': 489.569369}

In [ ]:
# Compute the power spectrum using the bias dictionary
Pkmu = eftobj.P_kmu_z(zz, f=like.cosmo.cosmo_dic['f_z'](zz), D=like.cosmo.cosmo_dic['D_z_k_func'](zz, 1e-5),
                      **biasdict_z1)
pkmu_bias = Pkmu(kvec, muvec)

# Project to multipoles
pk_gg = np.zeros((3,len(kvec)))
for i in range(3):
    integrand=np.array([pkmu_bias[j]*legendre(2*i)(muvec) for j in range(len(kvec))])
    pk_gg[i] = integ.simps(integrand, x=muvec, axis=1) * (4.*i+1)/2

In [ ]:
# Plot
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111)
ax.set_title(r'$z={0}$'.format(zz))
plt.plot(kvec, pk_gg[0], label=r'$P_0$')
plt.plot(kvec, pk_gg[1], label=r'$P_2$')
plt.plot(kvec, pk_gg[2], label=r'$P_4$')
ax.set_ylabel(r"$P_{gg,\ell}$")
ax.set_xlabel(r'$k$')
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend()
plt.show()